In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score

import numpy as np

In [3]:
iris = load_iris()

X = iris.data[:, :2]
y = iris.target

print(round(np.mean(cross_val_score(RandomForestClassifier(random_state=42), X, y, cv=3)), 4))

0.7132


#### Каков минимальный радиус круга, содержащего все точки из $X$?

In [4]:
from scipy.optimize import minimize

Первый способ

In [5]:
minimize(lambda point: ((point - X) ** 2).sum(axis=1).max(), np.mean(X, axis=0))

      fun: 3.468291571416955
 hess_inv: array([[ 0.00852046, -0.00986477],
       [-0.00986477,  0.0114212 ]])
      jac: array([2.39971387, 0.83110574])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 456
      nit: 3
     njev: 111
   status: 2
  success: False
        x: array([6.14927676, 3.16497268])

In [6]:
min_value = 1000000.
for x0 in np.linspace(6., 7., 10):
    for x1 in np.linspace(3., 4., 10):
        min_value = min(min_value, minimize(lambda point: ((point - X) ** 2).sum(axis=1).max(), np.array([x0, x1])).fun)

In [7]:
print(min_value)
print(round(np.sqrt(min_value), 4))

3.4648056830287155
1.8614


Второй способ

In [8]:
minimize(lambda point: ((point - X) ** 2)
         .sum(axis=1).max(), np.mean(X, axis=0), method='Nelder-Mead')

 final_simplex: (array([[6.15520434, 3.15154545],
       [6.15517458, 3.15160564],
       [6.15519425, 3.15154353]]), array([3.46483107, 3.46483507, 3.46484291]))
           fun: 3.4648310736449393
       message: 'Optimization terminated successfully.'
          nfev: 68
           nit: 36
        status: 0
       success: True
             x: array([6.15520434, 3.15154545])

In [9]:
min_value = minimize(lambda point: ((point - X) ** 2).sum(axis=1).max(), np.mean(X, axis=0), method='Nelder-Mead').fun
print(min_value)
print(round(np.sqrt(min_value), 4))

3.4648310736449393
1.8614


#### У вас есть набор зависимости $y$ от одной переменной $x$, причём вы знаете, что эта зависимость линейная ($y=ax + b$). Вам нужно восстановить эту зависимость, решив оптимизационную задачу $\min_{a, b} \sum_{i=1}^n (ax_i + b - y_i)^2$.

#### Проблема в том, что выборка очень большая, поэтому её нельзя полностью загрузить в память.

In [12]:
def generator():
    rand_gen = np.random.RandomState(42)
    for i in range(1000):
        arr_of_x = 10 * rand_gen.uniform(size=10000)
        arr_of_noise = 40 * rand_gen.normal(size=10000)
        for x, noise in zip(arr_of_x, arr_of_noise):
            yield x, 3.14 * x + 8.3 + noise

In [13]:
def solver(seq):
    count = 0
    sum_x = 0.
    sum_y = 0.
    sum_sq_x = 0.
    sum_xy = 0.
    for x, y in seq:
        count += 1
        sum_x += x
        sum_y += y
        sum_sq_x += x ** 2
        sum_xy += x * y
        
    a = (count * sum_xy - sum_x * sum_y) / (count * sum_sq_x - sum_x ** 2)
    b = (sum_y - a * sum_x) / count
    return a, b
        

In [14]:
%%time
a, b = solver(generator())
print('{}_{}'.format(round(a, 5), round(b, 5)))

3.13788_8.31801
CPU times: user 13.7 s, sys: 0 ns, total: 13.7 s
Wall time: 13.7 s
